Final project
20221399
Choi Hyungjin

cgcnn_test 폴더에 h.py 파일 생성 후 실행 (아래 코드 사용)
setup부터 import, download materials project data, train regression model까지의 과정을 포함

In [ ]:
import os
os. chdir("/home/edgpu/edgpu40/cgcnn_test/hands_on/train_sample")

import csv
import pandas as pd
from mp_api.client import MPRester

api_key = "KMqcsFzGZG8hOFbjBUBd9eJPVIojcV21"

def chunk_list(lst, chunk_size):
    """Split a list into smaller chunks."""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

with MPRester(api_key) as mpr:
    # First query to get material_id, composition, and band_gap from materials.summary
    summary_docs = mpr.materials.summary.search(
        chemsys="0-*-*-*-*-*",
        fields=["material_id", "composition", "band_gap"]
    ) # Hexanary oxides
    mpids = [doc.material_id for doc in summary_docs]

    print("Number of materials:", len(mpids))

    # Save the materials information to a CSV file
    with open("materials_info.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Material ID", "Composition", "Band Gap"])
        for doc in summary_docs:
            writer.writerow([doc.material_id, doc.composition, doc.band_gap])

    # Save the CIF files
    output_dir = "HexOx_cifs"
    os.makedirs(output_dir, exist_ok=True)

    for mpid in mpids:
        structure = mpr.materials.summary.search(material_ids=[mpid], fields=["structure"])[0].structure
        structure.to(fmt="cif", filename=os.path.join(output_dir, f"{mpid}.cif"))

# Process materials_info.csv to id_prop.csv
df = pd.read_csv("materials_info.csv")
df["Material ID"] = df["Material ID"].str.replace("mp-", "")
df_processed = df[["Material ID", "Band Gap"]]
df_processed.to_csv("id_prop.csv", header=False, index=False)

# Rename CIF files to remove "mp-" prefix
directory = "HexOx_cifs"
for filename in os.listdir(directory):
    if filename.startswith("mp-") and filename.endswith(".cif"):
        new_filename = filename.replace("mp-", "")
        old_file = os.path.join(directory, filename)
        new_file = os.path.join(directory, new_filename)
        os.rename(old_file, new_file)


그 후 id_prop_csv 파일과 atom_init.jsom파일을 생성된 HexOx_cifs 디렉토리에 옮기기

Train regression model - train
/gpfs/home/edgpu/edgpu40/cgcnn_test/hands_on/train_sample/ 디렉토리에 t.sh 파일 생성 후 실행 (아래 코드 사용)

In [ ]:
#!/bin/sh
#SBATCH -J cgcnn               # Job name
#SBATCH -p gpu_edu            # Partition name
#SBATCH -N 1                  # Number of nodes
#SBATCH -n 16                 # Number of CPU cores
#SBATCH -o %x.o%j             # Output log
#SBATCH -e %x.e%j             # Error log
#SBATCH --time=48:00:00       # Max wall time
#SBATCH --exclusive=user      # Exclusive node
#SBATCH --gres=gpu:2          # Request 2 GPUs

curr=`pwd`
cd "$curr"

module load conda/tensorflow-gpu

~/.conda/envs/cgcnn/bin/python ../../main.py \
    --train-ratio 0.6 \
    --val-ratio 0.2 \
    --test-ratio 0.2 \
    /gpfs/home/edgpu/edgpu40/cgcnn_test/HexOx_cifs/


Discussion part

1) Please figure out the schematic CGCNN architecture. Please have a look at main.py.

CGCNN은 crystal graph convolutional neural network의 줄임말로 결정 구조를 그래프로 변환한 후 graph convolution 연산을 통해 물성을 예측하는 neural network이다.

'main.py'를 통해 확인한 전반적인 학습 흐름은 다음과 같다.
1. 입력 처리
    'CIFData()'를 통해 '.cif' 파일을 로드하고 그래프로 변환
    각 원자는 'atom_init.json'을 통해 초기 feature vector로 변환됨
    이때 생성되는 입력은 'atom_features', 'neighbor_features', 'neighbor_idx', 'crystal_idx'가 됨
2. 데이터 분할
    `get_train_val_test_loader()` 함수로 train/val/test 세트 구성
    학습 시, batch 단위로 `train_loader`에서 데이터 받음
3. 모델 정의
    모델은 `CrystalGraphConvNet()` 클래스에서 정의됨
    하이퍼파라미터:
        `atom_fea_len` (기본: 64)
        `n_conv` (GCN layer 수, 기본: 3)
        `h_fea_len` (pooling 후 hidden layer 크기)
        `n_h` (fully connected hidden layer 수)
4. Graph Convolution Layer
    `ConvLayer` 클래스를 여러 번 반복 (n_conv)
    각 GCN layer는 원자(feature vector)를 이웃과의 관계(edge vector)를 통해 업데이트함
5. Pooling Layer
    전체 그래프(결정 구조)의 feature를 하나의 벡터로 요약
6. Fully Connected Layer
    Pooling된 feature를 여러 개의 hidden layer를 거쳐 출력층으로 보냄
    regression일 경우 bandgap을 예측 (`output_dim = 1`)
7. 학습 과정
    손실함수: `nn.MSELoss()` 또는 `nn.NLLLoss()`
    optimizer: `SGD` 또는 `Adam`
    scheduler: `MultiStepLR`로 학습률 감소
    평가: MAE (Mean Absolute Error)

여기서 사용된 주요 클래스와 함수는 다음과 같다.
- `CrystalGraphConvNet` (in `model.py`) : 핵심 모델 정의
- `CIFData` (in `data.py`) : 구조 파일 → 그래프 변환
- `train()` : 1 epoch 학습 수행
- `validate()` : 검증 또는 테스트 결과 평가
- `Normalizer` : 타겟 값의 평균/표준편차 정규화 및 복원

이를 시각적으로 표현하면 다음과 같다.
[CIF Files]
↓
[CIFData: convert to Graph]
↓
[GraphConvLayer 1] → [GraphConvLayer 2] → [GraphConvLayer 3] (n_conv)
↓
[Pooling (mean/sum)]
↓
[FC Layer 1] → [FC Layer 2] ... → [Output: Bandgap]

2) Please discuss the machine learning algorithm during the training. What is updated during the learning procedure?

CGCNN은 일반적인 딥러닝 모델과 마찬가지로 Backpropagation을 통해 모델의 parameter를 업데이트한다. 전체 학습의 흐름은 다음과 같다.
1. Forward propagation: 입력 구조를 모델에 넣어 예측값 계산  
2. Loss 계산: 예측값과 실제값 사이의 오차(loss) 계산  
3. Backward propagation: 오차에 대한 기울기(gradient) 계산  
4. Optimizer Step: 기울기를 기반으로 모델의 parameter(weight, bias 등) 업데이트

'main.py'파일의 'train()'함수를 통해 이를 확인할 수 있다. 그 부분은 다음과 같다.
    # compute output
    output = model(*input_var)
    loss = criterion(output, target_var)

    # measure accuracy and record loss
    if args.task == 'regression':
        mae_error = mae(normalizer.denorm(output.data.cpu()), target)
        losses.update(loss.data.cpu(), target.size(0))
        mae_errors.update(mae_error, target.size(0))
    else:
        accuracy, precision, recall, fscore, auc_score = \
            class_eval(output.data.cpu(), target)
        losses.update(loss.data.cpu().item(), target.size(0))
        accuracies.update(accuracy, target.size(0))
        precisions.update(precision, target.size(0))
        recalls.update(recall, target.size(0))
        fscores.update(fscore, target.size(0))
        auc_scores.update(auc_score, target.size(0))

    # compute gradient and do SGD step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
여기서 output = model(*input_var)은 예측값 계산, loss = criterion(output, target_var)은 손실 계산, 마지막 부분인 # compute gradient and do SGD step part는 Backpropagation을 담당한다.

이렇게 업데이트되는 항목들은 다음과 같다.
1. GCN Layer의 weight & bias: ConvLayer 내부의 가중치들이 업데이트됨
2. BatchNorm paremeter: 평균, 분산, 감마, 베타 등
3. Fully Connected Layer 가중치: Pooling 이후 MLP 층의 parameter
4. Optimizer 상태: 학습률, 모멘텀 값 등
5. Normalizer: 타겟 값의 평균/표준편차 - 학습 초기에 고정됨

3. Discuss the hyperparameters of this model and perform hyperparameter tuning. Discuss the most feasible way for this.

'main.py'를 통해 확인한 CGCNN의 주요 hyperparameter들은 다음과 같다.
| hyperparameter | 각 parameter 설명 | 기본값 |
|----------------|------|--------|
| `--batch-size` | 미니배치 크기 | 256 |
| `--lr` | 학습률 (learning rate) | 0.01 |
| `--epochs` | 학습 반복 횟수 | 30 |
| `--atom-fea-len` | GCN layer의 원자 feature 벡터 길이 | 64 |
| `--n-conv` | GCN layer의 층 수 | 3 |
| `--h-fea-len` | Pooling 후 hidden layer 크기 | 128 |
| `--n-h` | Fully connected hidden layer 개수 | 1 |
| `--optim` | 옵티마이저 (SGD 또는 Adam) | SGD |
| `--lr-milestones` | 학습률 조정 시점 (epoch 기준) | [100] |

이들이 각각 model에 미치는 영향은 다음과 같다.
    `batch-size`: 너무 작으면 noisy, 너무 크면 overfit 또는 느림
    `lr`: 너무 크면 발산, 너무 작으면 느림 → 학습률 스케줄러 사용
    `n-conv`: GCN 레이어 수, 너무 많으면 over-smoothing 가능
    `atom-fea-len`, `h-fea-len`: feature 벡터의 크기, 모델 표현력 결정
    `optimizer`: 일반적으로 Adam이 더 빠르게 수렴

hyperparameter tuning을 통해 학습의 안정성을 조절할 수 있으며 그 방법은 위에서 사용한 t.sh파일을 수정하는 것이다. 예를 들어, 
#!/bin/sh
#SBATCH -J cgcnn               # Job name
#SBATCH -p gpu_edu            # Partition name
#SBATCH -N 1                  # Number of nodes
#SBATCH -n 16                 # Number of CPU cores
#SBATCH -o %x.o%j             # Output log
#SBATCH -e %x.e%j             # Error log
#SBATCH --time=48:00:00       # Max wall time
#SBATCH --exclusive=user      # Exclusive node
#SBATCH --gres=gpu:2          # Request 2 GPUs

curr=`pwd`
cd "$curr"

module load conda/tensorflow-gpu

~/.conda/envs/cgcnn/bin/python /gpfs/home/edgpu/edgpu40/cgcnn_test/main.py \
    --train-ratio 0.6 \
    --val-ratio 0.2 \
    --test-ratio 0.2 \
    --batch-size 256 \
    --lr 0.005 \
    /gpfs/home/edgpu/edgpu40/cgcnn_test/HexOx_cifs/

등으로 수정할 수 있다.

여러 parameter을 수정하며 실험한 결과는 다음과 같다.
| 실험 번호 | batch-size | lr   | MAE (val) |
|-----------|------------|------|------------|
| 1(기본값)  | 256        | 0.01 | 0.584      |
| 2         | 256        | 0.005| 0.588      |
| 3         | 256        | 0.02 | 0.713      |
| 4         | 128        | 0.01 | 0.560      |
| 5         | 64         | 0.01 | 0.596      |
우선, 다른 조건은 유지하고 learning rate(lr)을 수정하였다. 그 결과 lr을 0.01에서 0.005로 낮췄을 때의 성능 차이는 거의 없었으며 lr을 0.02로 높였을 때 MAE값이 0.713으로 더 높았다.
다음으로 batch-size를 수정하였다. 그 결과 batch-size를 128로 줄였을 때 MAE값이 낮아졌지만 256인 경우와 차이가 별로 없었다. batch-size를 64로 줄였을 경우 MAE값이 높아졌으며 이는 지나치게 작은 batch-size로 인해 학습이 불안정해졌다고 이해할 수 있다. 하지만 이 역시 batch-size가 256인 값과 크게 차이가 나지 않는다.
이를 통해 batch-size는 모델의 성능에 큰 영향을 미치지 않지만 lr은 큰 영향을 미칠 수 있다는 점을 알 수 있다.
이 외에도 향후 layer의 수, feature size등을 조절해보며 성능을 확인해보면 좋을 것 같다.

이러한 hyperparameter를 적절한 값으로 튜닝하는 방법으로는 여러 가지가 있다.
첫 번째로, Grid Search가 있다. 이는 여러 조합을 미리 정의하고 실험하는 방법이다.
다음으로 Random Search가 있다. 이는 조합을 무작위로 추출하여 실험하는 방법이다.
가장 현실적인 방법으로는 Manual Search가 있으며 이는 몇 가지 조합을 바꿔가며 결과를 확인하는 방법이다.

4. Plot the ML performance with respect to the epoch. What epoch number is enough?

CGCNN 모델의 학습 epoch 수에 따른 MAE값을 확인하기 위해 3번의 parameter를 수정하는 방법과 같이 epoch를 5, 15, 30(기본값), 50, 75로 수정하며 실험을 진행하였다. 그 결과는 다음과 같다.
| 실험 번호 | epoch 수 | MAE (val) |
|-----------|----------|---------|
| 1         | 5        | 0.821   |
| 2         | 15       | 0.822   |
| 3         | 30       | 0.584   |
| 4         | 50       | 0.568   |
| 5         | 75       | 0.531   |
이를 통해 epoch 수가 작을 경우(5, 15) 모델이 거의 학습되지 않아 MAE가 매우 크게 나오는 것을 알 수 있다. epoch 수가 30 부터 MAE값이 크게 감소하여 0.584에 이르렀다. 그 후 epoch 수가 늘어날 수록 MAE값이 감소하는 경향을 보였지만, 그 감소폭은 점점 줄어들었다. 특히 epoch=30인 경우와 그의 두배가 넘는 epoch=75인 경우의 MAE값이 0.053밖에 나지 않는 것을 통해 학습 시간을 고려해 epoch=30이 가장 적절한 수로 판단된다.

이를 그래프로 나타내기 위해 아래 코드를 사용할 수 있다.

In [ ]:
import matplotlib.pyplot as plt

epochs = [5, 15, 30, 50, 75]
mae = [0.821, 0.822, 0.584, 0.568, 0.531]

plt.plot(epochs, mae, marker='o')
plt.xlabel("Epochs")
plt.ylabel("Validation MAE")
plt.title("Validation MAE vs Epochs")
plt.grid(True)
plt.tight_layout()
plt.show()

5. Please adjust the frequency to print out the step. What is the difference between epoch and step?

Epoch는 전체 학습 데이터셋을 모델이 '한 번 학습'한 횟수를 의미한다. 즉 Epoch값이 클 수록 학습 횟수가 증가하는 것이다. 반면 Step은 Iteration으로도 불리며 이는 한 mini-batch를 갖고 학습한 횟수를 의미한다. 즉, epoch와 step 사이에는 1 epoch = (전체 데이터 수 / batch size) * step 의 관계식이 성립한다.

'main.py'에서 확인한 결과 "print-freq', '-p', default=10"을 통해 print frequency가 10으로 고정되어 있음을 알 수 있다. 이는 10step마다 현재 상태를 출력하는 것을 의미하며 이 값을 조절하여 로그의 출력 빈도를 조절할 수 있다.

6. What is the role of batch size, and how can you rationalize the number and size of batches quantitatively?

Batch size란 한 번의 step에서 모델이 학습에 사용하는 데이터 샘플의 수를 의미한다. 
ML에서 Batch size는 많은 역할을 한다. 첫 번째로 메모리 사용량을 결정하며 batch size가 클수록 더 많은 메모리를 필요로 한다. 따라서 딥러닝에서 전체 데이터를 한 번에 사용하는 Full batch는 메모리 효율이 떨어진다.
다음으로 학습 속도에 영향을 미친다. 너무 적은 샘플만 사용하는 Mini batch는 불안정한 학습을 유발하여 moise가 생길 수 있으며 반복이 많아 시간이 오래 걸린다. 이 외에도 graident 추정 정확도와 regularization 등의 효과를 낼 수 있다. 따라서 적절한 batch size를 선택하는 것이 중요하다.

위 3번의 실험 결과를 통해 batch size가 매우 낮을 경우 모델의 성능은 많이 떨어지지만, 일정 수준 이상이 되면 모델 성능의 차이가 크지 않다는 것을 알 수 있다. 따라서 직접 실험을 통해 일정 수준까지만 batch size를 높이는 것이 좋다고 생각된다.

7. Please verify the roles of the atom_init.json and id_prop.csv files. State the definition of the atom_init.json file. Explain how it generates nodes and what information it inclues. Additionally, describe how the nodes are updated during each epoch in the training process.

'atom_init.json'은 원자 초기 feature 벡터를 정의한다. 이 파일은 각 원소에 대해 고정된 feature vector를 부여한 json 형식의 딕셔너리이다. 여기서 key는 원자 번호이고 value는 벡터를 의미한다. 일반적으로 약 64차원의 벡터가 사용되며 이 벡터들은 atom_fea_len 하이퍼파라미터에 의해 결정되고 학습의 초기값으로만 사용된 후 학습 중 업데이트되지는 않는다.
이 파일은 .cif파일을 로드하여 결정 구조를 그래프로 변환하는 과정에서 각 원자 번호에 해당하는 벡터의 저장소로 사용된다. 그 벡터는 그래프에서 각 노드의 초기 feature로 사용된다.
이후 atom_init.json 자체는 업데이트 되지 않지만, 그 벡터들로 구성된 노드 feature는 GraphConv Layer를 통해 계속해서 업데이트된다. 여기서 ConvLayer는 각 원자의 이웃 노드, 이웃과의 거리 등의 엣지 정보를 활용하여 노드 feature를 업데이트한다.

'id_prop.csv'파일은 각 .cif파일에 해당하는 구조와 band gap와 같이 그에 대응하는 타겟 물성 값을 정의한 csv파일이다. 여기서 첫 번째 열은 파일명을 의미하며 두 번째 열은 예측할 값인 타겟을 의미한다. 

즉, 전체 학습의 흐름을 요약하면 다음과 같다.
[.cif file]
   ↓
[atom_init.json] → 노드 초기화 (feature vector)
[id_prop.csv]    → 예측 대상 설정 (label)
   ↓
[CIFData → Graph로 변환]
   ↓
[GraphConv Layer → feature 업데이트]
   ↓
[Pooling → Fully Connected Layer → 예측]
   ↓
[loss 계산 → 파라미터 업데이트]

8. What is the role of the validation set?

ML에서는 사용하는 데이터셋은 Train set, Validation set, Test set이 있다.
첫 단계인 Train set은 모델의 parameter을 학습시키는 데 사용되는 데이터셋이다.
다음 단계인 Validation set은 학습 과정 중 모델의 일반화 성능을 평가하기 위한 데이터셋이다. 이는 학습 자체에는 사용되지 않으며 중간 성능 측정 과정에만 사용되며 hyperparameter을 조정하며 과적합을 감지하는 역할을 한다. 또한 val MAE의 개선이 없을 경우 학습을 중단하는 기준이 된다.
마지막 단계인 Test set은 최종 모델 성능을 평가하기 위한 독립적인 데이터셋이다.

'main.py'에서는 mae_error = validate(val_loader, model, criterion, normalizer)를 통해 매 epoch마다 `validate()` 함수를 통해 validation MAE를 계산한다.

일반적으로 train:val:test = 6:2:2 또는 8:1:1의 조합을 사용하여 모델 튜닝을 통해 신뢰도를 높일 수 있으며 여기서는 6:2:2의 조합을 사용하였다.

9. Please discuss the model’s performance, what do you think about the regression result? Please discuss your idea on this result.

모델의 performance를 확인하기 위해 Predicted Band Gap vs. True Band Gap plot를 그려보았다. 그 그래프는 다음과 같다.
![Prediction vs True](prediction_vs_true.jpg)
이 그래프의 대부분의 점들이 y=x 직선의 근처에 위치하고 있는 사실을 통해 전반적으로 band gap을 잘 예측하고 있음을 알 수 있다. 다만 band gap이 0에 가까운 물질들에 대해 예측 오차가 다소 크게 나타났으며 일부는 음수로 예측되기도 하였다. 이는 모델의 고질적인 한계점에 속한다.
이를 통헤 전반적으로 CGCNN모델은 구조 정보만으로 band gap을 높은 정확도로 예측하고 있으며 이는 photocatalyst의 후보 탐색과 같은 실험 설계 단계에서 매우 유용하게 활용될 것 같다.

이를 위해 /gpfs/home/edgpu/edgpu40/cgcnn_test/hands_on/train_sample/ 디렉토리에 p.py 파일 생성 후 실행하여 plot.jpg파일을 얻었다. p.py 코드로는 아래 코드를 사용하였다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 서버용 백엔드 설정
import matplotlib
matplotlib.use('Agg')

# CSV 읽기
df = pd.read_csv('test_results.csv', header=None)
df.columns = ['Material ID', 'True', 'Predicted']

# 산점도 그리기
plt.figure(figsize=(6, 6))
plt.scatter(df['True'], df['Predicted'], alpha=0.7)
plt.plot([df['True'].min(), df['True'].max()],
         [df['True'].min(), df['True'].max()],
	 'r--')

plt.xlabel('True Band Gap')
plt.ylabel('Predicted Band Gap')
plt.title('CGCNN Prediction vs. Ground Truth')
plt.grid(True)
plt.tight_layout()

# 이미지로 저장
plt.savefig('prediction_vs_true.jpg', dpi=300)


10. Please do your project via Jupyter notebook. Upload your file (including the result, data file…) in github. Your code must be reproduced.